In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib
import hdbscan

from sklearn.datasets import load_digits
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_openml
import sklearn.datasets as dt
import scipy

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist
from sklearn.utils import resample
from tensorflow.keras.datasets.fashion_mnist import load_data

#Import all the algorithms
from umap import UMAP
import openTSNE
from openTSNE import TSNE as OpenTSNE
from openTSNE import affinity, initialization, TSNEEmbedding

import time

In [ ]:
def embedding_metrics(X, Z, classes, knn=10, knn_classes=10, subsetsize=1000):
    intersections_knn = 0
    intersections_knc = 0

    ## KNN ##
    #Compute the set of KNN on the true and embedded data
    neighbor_true_data_knn = NearestNeighbors(n_neighbors=knn).fit(X)
    set_true_knn = set(neighbor_true_data_knn.kneighbors(return_distance=False))
    neighbor_embed_data_knn = NearestNeighbors(n_neighbors=knn).fit(Z)
    set_embed_knn = set(neighbor_embed_data_knn.kneighbors(return_distance=False))

    #Compute the intersections between true and embedding
    for i in range(X.shape[0]):
        intersections_knn += len(set_true_knn[i] & set_embed_knn[i])
    KNN = intersections_knn / (X.shape[0] * knn)

    ## KNC ##
    #Build the class means
    cl, cl_inv = np.unique(classes, return_inverse=True)
    C = cl.size
    mu1 = np.zeros((C, X.shape[1]))
    mu2 = np.zeros((C, Z.shape[1]))
    for c in range(C):
        mu1[c,:] = np.mean(X.iloc[cl_inv==c,:], axis=0)
        mu2[c,:] = np.mean(Z[cl_inv==c,:], axis=0)
    
    #KNN on the class means
    neighbor_true_data_knc = NearestNeighbors(n_neighbors=knn_classes).fit(mu1)
    set_true_knc = neighbor_true_data_knc.kneighbors(return_distance=False)
    neighbor_embed_data_knc = NearestNeighbors(n_neighbors=knn_classes).fit(mu2)
    set_embed_knc = neighbor_embed_data_knc.kneighbors(return_distance=False)
    
    #Compute the intersection between true and embedding
    for i in range(C):
        intersections_knc += len(set(set_true_knc[i]) & set(set_embed_knc[i]))
    KNC = intersections_knc / (C * knn_classes)
    
    ## CPD ##
    #Pick a subset of pairwise points
    subset = np.random.choice(X.shape[0], size=subsetsize, replace=False)
    distance_true = pdist(X[subset,:])
    distance_embed = pdist(Z[subset,:])
    CPD = scipy.stats.spearmanr(distance_true[:,None],
            distance_embed[:,None]).correlation
    
    return KNN, KNC, CPD

In [ ]:
#Load MNIST Fashion
fashion = fetch_openml('Fashion-MNIST', version=1)
X = fashion.data
y = fashion.target.astype('int')
#PCA
X_whitened = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_whitened, full_matrices=False)
X784 = np.dot(U, np.diag(s))[:,:784]
n = X784.shape[0]n = X784.shape[0]

In [ ]:
embed =[]
# Methods
#PCA - 2 components
%time embed.append(PCA(n_components=2).fit_transform(X))
#t-SNE random initialization
%time embed.append(OpenTSNE(n_jobs=-1, random_state=42,
        initialization='random',negative_gradient_method="bh").fit(X784))
#t-SNE PCA initialization
%time embed.append(OpenTSNE(n_jobs=-1, random_state=42,
        negative_gradient_method="bh").fit(X784))
#Multi-Scale kernel
%time affinities_multiscale_mix = affinity.Multiscale(X784, 
        perplexities=[30, int(X784.shape[0]/100)], n_jobs=-1, random_state=42)
%time pca_init = initialization.pca(X, 
        random_state=42)
%time embedding0 = TSNEEmbedding(pca_init,affinities_multiscale_mix,
        negative_gradient_method="bh",n_jobs=8,random_state=42)
%time embedding1 = embedding0.optimize(n_iter=250, exaggeration=12,
        momentum=0.5,random_state=42)
%time embed.append(embedding1.optimize(n_iter=750, exaggeration=1, 
        momentum=0.5, random_state=42))
#Learning Rate adjusted
%time affinities_multiscale_mix = affinity.Multiscale(X784, 
        perplexities=[30, int(X784.shape[0]/100)], n_jobs=-1, random_state=42)
%time pca_init = initialization.pca(X, random_state=42)
%time embedding0 = TSNEEmbedding(pca_init,affinities_multiscale_mix,n/12,
        negative_gradient_method="bh",n_jobs=8,random_state=42)
%time embedding1 = embedding0.optimize(n_iter=250, exaggeration=12, 
        momentum=0.5,random_state=42)
%time embed.append(embedding1.optimize(n_iter=750, exaggeration=1, 
        momentum=0.5,random_state=42))
#UMAP random initialization
%time embed.append(UMAP(random_state=42, init='random').fit_transform(X))
#UMAP LE initialization
%time embed.append(UMAP(random_state=42).fit_transform(X))

In [ ]:
%%time
#Calculate KNN, KNC, and CPD
metrics = []
for i in range(len(embed)):
    knn, knc, cpd = embedding_metrics(X, embed[i], y, knn=10, knn_classes=4, subsetsize=1000)
    metrics.append((knn, knc, cpd))

In [ ]:
titles = ["PCA", "(Fi)t-SNE with PCA initialization\n(Learning rate $\eta=200$ Perplexity 30)", 
          "(Fi)t-SNE with PCA initialization\n(Multiscale Similarities 30, n/100)",
          "(Fi)t-SNE with PCA initialization\n(Learning rate $\eta=n/12$\nMultiscale Similarities 30, n/100)", "UMAP with Random Initialization",
          "UMAP with Laplacian Eigenmaps"]

letters = 'abcdefgh'

plt.figure(figsize=(7.2, 4.5))

# Add plots 
for i,Z in enumerate(embed):
    plt.subplot(2,3,1+i)
    plt.gca().set_aspect('equal', adjustable='datalim')
    rand_order = np.random.permutation(Z.shape[0])
    plt.scatter(Z[rand_order,0], Z[rand_order,1], s=1, c=y[rand_order], cmap='rainbow', 
                rasterized=True, edgecolor='none')
    plt.title(titles[i], va='center')
    plt.text(0.69,.02,'KNN:\nKNC:\nCPD:', transform=plt.gca().transAxes, fontsize=6)
    plt.text(0.90,.02,'{:.2f}\n{:.2f}\n{:.2f}'.format(
        metrics[i][0], metrics[i][1], metrics[i][2]), transform=plt.gca().transAxes, fontsize=6)
    plt.text(0, 0, letters[i], transform = plt.gca().transAxes, fontsize=8, fontweight='bold')
    plt.xticks([])
    plt.yticks([])
    
sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('clustering/mnist-fash-simulation.png', dpi=600)

In [ ]:
#Prepare data to cluster
embeddings_to_cluster = [X]
for Z in embed:
    embeddings_to_cluster.append(Z)

clustering_labels = []
clustered_labels = []
rand_score_clustered = []
mi_score_clustered = []
rand_score = []
mi_score = []
total_capture = []

#Cluster based on all embeddings including the raw data
for i in range(len(embeddings_to_cluster)):
    if i == 0:
        clustering_labels.append(hdbscan.HDBSCAN(min_samples=10,
            min_cluster_size=500).fit_predict(X784))
    else:
        clustering_labels.append(hdbscan.HDBSCAN(min_samples=10,
            min_cluster_size=500).fit_predict(embeddings_to_cluster[i]))
    clustered = (clustering_labels[i] >= 0)
    clustered_labels.append(clustered)
    #Total Score
    rand_score.append(adjusted_rand_score(y, clustering_labels[i]))
    mi_score.append(adjusted_mutual_info_score(y, clustering_labels[i]))
    #Scores based on what was actually clustered
    rand_score_clustered.append(adjusted_rand_score(y[clustered],
                                     clustering_labels[i][clustered]))
    mi_score_clustered.append(adjusted_mutual_info_score(y[clustered], 
                                     clustering_labels[i][clustered]))
    #Total capture
    total_capture.append(np.sum(clustered) / X.shape[0])

#Add best KNN/KNC/CPD trade off embedding as benchmark for HDBSCAN
embed_for_cluster = [embed[1]]
for Z in embed:
    embed_for_cluster.append(Z)

In [ ]:
titles = ["HDBSCAN", "PCA", "(Fi)t-SNE with PCA initialization\n(Learning rate $\eta=200$ Perplexity 30)", 
          "(Fi)t-SNE with PCA initialization\n(Multiscale Similarities 30, n/100)",
          "(Fi)t-SNE with PCA initialization\n(Learning rate $\eta=n/12$\nMultiscale Similarities 30, n/100)", "UMAP with Random Initialization",
          "UMAP with Laplacian Eigenmaps"]

letters = 'abcdefgh'

plt.figure(figsize=(7.2, 4.5))

#Plot clustering on top of the embedding
for i,Z in enumerate(embed_for_cluster):
    plt.subplot(2,4,1+i)
    plt.gca().set_aspect('equal', adjustable='datalim')
    #PLot clustering on top of the non-clustered data
    plt.scatter(Z[~clustered_labels[i],0], Z[~clustered_labels[i],1], 
                s=1, color=(.5,.5,.5), alpha=0.5,
                rasterized=True, edgecolor='none') #not clustered
    plt.scatter(Z[clustered_labels[i],0], Z[clustered_labels[i],1], s=1, 
                c=clustering_labels[i][clustered_labels[i]], cmap='rainbow', 
                rasterized=True, edgecolor='none') #clustered
    plt.title(titles[i], va='center')
    #Add clustering scores
    plt.text(0.69,.02,'MIC:\nARIC:\nMI:\nARI:', transform=plt.gca().transAxes, fontsize=6)
    plt.text(0.90,.02,'{:.2f}\n{:.2f}\n{:.2f}\n{:.2f}'.format( 
        mi_score_clustered[i], rand_score_clustered[i], mi_score[i], rand_score[i]),
        transform=plt.gca().transAxes, fontsize=6)
    #Add capture %
    plt.text(0.1,0.9, 'C%:', transform = plt.gca().transAxes, fontsize=8, fontweight='bold')
    plt.text(0.28,0.9, '{:.2f}'.format(total_capture[i]*100), transform = plt.gca().transAxes, fontsize=8, fontweight='bold')
    plt.text(0, 0, letters[i], transform = plt.gca().transAxes, fontsize=8, fontweight='bold')
    plt.xticks([])
    plt.yticks([])

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('clustering/report/cluster_fash.png', dpi=600)